In [2]:
import pandas as pd
import celib
from celib import MRE
from celib import DB1
import numpy as np
import sys
import os

In [3]:
os.chdir('c:\\Users\\ib400\\OneDrive - University of Exeter\\Desktop\\PhD\\GitHub\\FTT_StandAlone')

In [5]:
# Get titles
with DB1(r'Emulation\data\U.db1') as dbu:
    regions = pd.read_csv(r"Emulation\data\e3me_regions_jan23.csv", header=None)[0].values.tolist()   # Region titles from U.db1
    regions =  [s.strip() for s in regions]
    regions_short = dbu['RSHORTTI']
    fuel = dbu['FUTI']
    power = dbu["T2TI"]
    power_erti = dbu["ERTI"]
    tr = dbu["VTTI"]
    sec_eu = dbu["QTI"]
    sec = dbu["QRTI"]
print(power_erti)

['1 Nuclear', '2 Oil', '3 Coal', '4 Gas', '5 Biomass', '6 Biogas', '7 Biogas + CCS', '8 Tidal', '9 Large Hydro', '10 Onshore', '11 Offshore', '12 Solar PV', '13 Geothermal', '14 Wave']


In [7]:
years = range(2010, 2071)
results = pd.DataFrame()

mre_list = ["Dan_1-5c"]
for m in mre_list:
    # Load mre
    print("Processing: " + str(m))
    mre = MRE("Emulation/data/" + str(m) + ".mre")
    ex = pd.DataFrame(mre["EX"][0], index=regions, columns=years)
    prsc = pd.DataFrame(mre["PRSC"][0], index=regions, columns=years)
    defl = prsc/ex
    repp = pd.DataFrame((mre['REPP'][0]/(44/12)*1.33), index=regions, columns=years)
    repp_adj = repp/defl
    mewd = pd.DataFrame(mre['MEWD'][:][7][:], index=regions, columns=years)
repp_adj.to_excel("repp_adj.xlsx")
#mewd.to_excel("mewd.xlsx")

Processing: Dan_1-5c


In [60]:
mewd = np.array(mre['MEWD'])
pd.DataFrame(mewd[:,7,:], index=regions, columns=years).to_excel("mewd_electricity_S3.xlsx")


In [53]:
mewd = np.array(mre['MEWD'])
mewd[:,7,:].shape


(71, 61)

In [6]:
# Set list of mre files
mre_list = ["SUP_NPi_Default", "SUP_1p5C_Default", ]
# mre_list = ["SUP_NPi_Default", "SUP_2C_Default", "SUP_1p5C_Default",
#            "SUP_2C_Lab", "SUP_1p5C_Lab", "SUP_2C_Lump", "SUP_1p5C_Lump",
#            "SUP_2C_VAT", "SUP_1p5C_VAT","SUP_2C_Regional",
#            "SUP_1p5C_Regional"]
# mre_list = ["SUP_NPi_Default", "SUP_2C_Default", "SUP_1p5C_Default"]
years = range(2010, 2051)
results = pd.DataFrame()

for m in mre_list:
    # Load mre
    print("Processing: " + str(m))
    mre = MRE("C:/E3ME/EEIST/Output/" + str(m) + ".mre")
    
    # 2d variables
    # Aggregate GDP (+ conversion)
    rgdp = pd.DataFrame((mre['RGDP'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rgdp["Variable"] = "GDP|MER"
    rgdp["Unit"] = "billion US$2010/yr"
    
    # Population (+ conversion)
    rpop = pd.DataFrame((mre['RPOP'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rpop["Variable"] = "Population"
    rpop["Unit"] = "million"
    
    # Aggregate gov consumption (+ conversion)
    rsg = pd.DataFrame((mre['RSK'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsg["Variable"] = "Investment"
    rsg["Unit"] = "billion US$2010/yr"
    
    # Aggregate investment (+ conversion)
    rsk = pd.DataFrame((mre['RSG'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsk["Variable"] = "Expenditure|Government"
    rsk["Unit"] = "billion US$2010/yr"
    
    # Aggregate household consumption (+ conversion)
    rsc = pd.DataFrame((mre['RSC'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsc["Variable"] = "Expenditure|Household"
    rsc["Unit"] = "billion US$2010/yr"
    
    # Aggregate export (+ conversion)
    rsx = pd.DataFrame((mre['RSX'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsx["Variable"] = "Export"
    rsx["Unit"] = "billion US$2010/yr"
    
    # Aggregate import (+ conversion)
    rsm = pd.DataFrame((mre['RSM'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsm["Variable"] = "Import"
    rsm["Unit"] = "billion US$2010/yr"
    
    # Aggregate employment (+ conversion)
    remp = pd.DataFrame((mre['REMP'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    remp["Variable"] = "Employment"
    remp["Unit"] = "million"    
    
    # Aggregate unemployment (+ conversion)
    rune = pd.DataFrame((mre['RUNE'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rune["Variable"] = "Unemployment"
    rune["Unit"] = "million"
    
    # Cabron price (+ conversion)
    repp = pd.DataFrame((mre['REPP'][0]/(44/12))*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    repp["Variable"] = "Price|Carbon"
    repp["Unit"] = "US$2010/t CO2"
        
    
    # 3d variables
    # CO2 emissions (+ conversion)
    fco2 = {}
    fco2_e = {}
    # Power generation
    mewk = {}
    mewk_tot = {}
    mewk_bccs = {}
    mewk_b = {}
    mewk_btot = {}
    mewk_cccs = {}
    mewk_c = {}
    mewk_ctot = {}
    mewk_gccs = {}
    mewk_g = {}
    mewk_gtot = {}
    mewk_hy = {}
    mewk_h = {}
    mewk_n = {}
    mewk_o = {}
    mewk_scsp = {}
    mewk_spv = {}
    mewk_stot = {}
    mewk_w = {}
    mewk_wof = {}
    mewk_won = {}
    mewk_geo = {}
    # Primary Energy
    rery = {}
    rery_tot = {}
    rery_c = {}
    rery_g = {}
    rery_o = {}
    rery_b = {}
    rery_s = {}
    rery_w = {}
    rery_geo = {}
    rery_h = {}
    rery_n = {}
    # Secondary Energy
    fret = {}
    
    for r, reg in enumerate(regions):
        fco2[reg] = pd.DataFrame((mre['FCO2'][r])/(3/11)/1000, index=fuel, columns=years)
        # Get just energy emissions
        fco2_e[reg] = fco2[reg].drop(["8 Ore-extra.(non-energy)", "23 Non-energy use"])
        
        # get sums of emissions across fuel users
        fco2[reg] = fco2[reg].sum()
        fco2_e[reg] = fco2_e[reg].sum()
        
        # MEWK
        mewk[reg] = pd.DataFrame(mre["MEWK"][r], index=power, columns=years)
        # Total electricty capacity
        mewk_tot[reg] = mewk[reg].sum()
        # Biomass w/ccs
        mewk_bccs[reg] = mewk[reg].iloc[[9,11,13]].sum()
        # Biomass
        mewk_b[reg] = mewk[reg].iloc[[8, 10, 12]].sum()
        # Biomass TOTAL
        mewk_btot[reg] = mewk[reg].iloc[[8,9,10,11,12,13]].sum()
    
        # Coal w/ccs
        mewk_cccs[reg] = mewk[reg].iloc[3]
        # Coal
        mewk_c[reg] = mewk[reg].iloc[2]
        # Coal TOTAL
        mewk_ctot[reg] = mewk[reg].iloc[[2,3]].sum()
        
        # Gas w/ccs
        mewk_gccs[reg] = mewk[reg].iloc[[5, 7]].sum()
        # Gas
        mewk_g[reg] = mewk[reg].iloc[[4, 6]].sum()
        # Gas TOTAL
        mewk_gtot[reg] = mewk[reg].iloc[[4,5,6,7]].sum()
        
        # Hydro
        mewk_hy[reg] = mewk[reg].iloc[[14, 15, 21]].sum()
        # Nuclear
        mewk_n[reg] = mewk[reg].iloc[0]
        # Oil 
        mewk_o[reg] = mewk[reg].iloc[1]
        # CSP 
        mewk_scsp[reg] = mewk[reg].iloc[19]
        # Solar pv
        mewk_spv[reg] = mewk[reg].iloc[18]
        # Solar TOTAL
        mewk_stot[reg] = mewk[reg].iloc[[18,19]].sum()

        # Total wind
        mewk_w[reg] = mewk[reg].iloc[[16, 17]].sum()
        # Offshore wind
        mewk_wof[reg] = mewk[reg].iloc[17]
        # Onshore wind
        mewk_won[reg] = mewk[reg].iloc[16]
        # Geothermal
        mewk_geo[reg] = mewk[reg].iloc[20]
        
        # RERY
        rery[reg] = pd.DataFrame((mre["RERY"][r])/1000, index=power_erti, columns=years)
        # Total electricty capacity
        rery_tot[reg] = rery[reg].sum()
        # Coal
        rery_c[reg] = rery[reg].iloc[[2]].sum()
        # Gas
        rery_g[reg] = rery[reg].iloc[[3]].sum()
        # Oil
        rery_o[reg] = rery[reg].iloc[[1]].sum()
        # Biomass
        rery_b[reg] = rery[reg].iloc[[4,5,6]].sum()
        # Solar
        rery_s[reg] = rery[reg].iloc[[11]].sum()
        # Wind
        rery_w[reg] = rery[reg].iloc[[9,10]].sum()
        # Geothermal
        rery_geo[reg] = rery[reg].iloc[[12]].sum()
        # Hydro
        rery_h[reg] = rery[reg].iloc[[7,8,13]].sum()        
        # Nuclear
        rery_n[reg] = rery[reg].iloc[[0]].sum()       
        
        # FRET
        fret[reg] = pd.DataFrame((mre['FRET'][r])*1000*4.1868E-08, index=fuel, columns=years)
        fret[reg] = fret[reg].sum()
        
    
    fco2 = pd.DataFrame.from_dict(fco2, orient='index').reset_index().rename(columns={"index":"Region"})
    fco2["Variable"] = "Emissions|CO2"
    fco2["Unit"] = "Mt CO2/yr"
    
    fco2_e = pd.DataFrame.from_dict(fco2_e, orient='index').reset_index().rename(columns={"index":"Region"})
    fco2_e["Variable"] = "Emissions|CO2|Energy"
    fco2_e["Unit"] = "Mt CO2/yr"
    
    mewk_tot = pd.DataFrame.from_dict(mewk_tot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_tot["Variable"] = "Capacity|Electricity"
    mewk_tot["Unit"] = "GW"
    
    mewk_bccs = pd.DataFrame.from_dict(mewk_bccs, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_bccs["Variable"] = "Capacity|Electricity|Biomass|w/ CCS"
    mewk_bccs["Unit"] = "GW"    

    mewk_b = pd.DataFrame.from_dict(mewk_b, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_b["Variable"] = "Capacity|Electricity|Biomass|w/o CCS"
    mewk_b["Unit"] = "GW"    

    mewk_btot = pd.DataFrame.from_dict(mewk_btot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_btot["Variable"] = "Capacity|Electricity|Biomass"
    mewk_btot["Unit"] = "GW"        
    
    mewk_cccs = pd.DataFrame.from_dict(mewk_cccs, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_cccs["Variable"] = "Capacity|Electricity|Coal|w/ CCS"
    mewk_cccs["Unit"] = "GW"    
 
    mewk_c = pd.DataFrame.from_dict(mewk_c, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_c["Variable"] = "Capacity|Electricity|Coal|w/o CCS"
    mewk_c["Unit"] = "GW"    

    mewk_ctot = pd.DataFrame.from_dict(mewk_ctot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_ctot["Variable"] = "Capacity|Electricity|Coal"
    mewk_ctot["Unit"] = "GW"        
    
    mewk_gccs = pd.DataFrame.from_dict(mewk_gccs, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_gccs["Variable"] = "Capacity|Electricity|Gas|w/ CCS"
    mewk_gccs["Unit"] = "GW" 

    mewk_g = pd.DataFrame.from_dict(mewk_g, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_g["Variable"] = "Capacity|Electricity|Gas|w/o CCS"
    mewk_g["Unit"] = "GW" 
    
    mewk_gtot = pd.DataFrame.from_dict(mewk_gtot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_gtot["Variable"] = "Capacity|Electricity|Gas"
    mewk_gtot["Unit"] = "GW"     
    
    mewk_hy = pd.DataFrame.from_dict(mewk_hy, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_hy["Variable"] = "Capacity|Electricity|Hydro"
    mewk_hy["Unit"] = "GW"    
    
    mewk_n = pd.DataFrame.from_dict(mewk_n, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_n["Variable"] = "Capacity|Electricity|Nuclear"
    mewk_n["Unit"] = "GW"        
    
    mewk_o = pd.DataFrame.from_dict(mewk_o, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_o["Variable"] = "Capacity|Electricity|Oil|w/o CCS"
    mewk_o["Unit"] = "GW"   
    
    mewk_scsp = pd.DataFrame.from_dict(mewk_scsp, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_scsp["Variable"] = "Capacity|Electricity|Solar|CSP"
    mewk_scsp["Unit"] = "GW"       
    
    mewk_spv = pd.DataFrame.from_dict(mewk_spv, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_spv["Variable"] = "Capacity|Electricity|Solar|PV"
    mewk_spv["Unit"] = "GW"        

    mewk_stot = pd.DataFrame.from_dict(mewk_stot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_stot["Variable"] = "Capacity|Electricity|Solar"
    mewk_stot["Unit"] = "GW"    
    
    mewk_w = pd.DataFrame.from_dict(mewk_w, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_w["Variable"] = "Capacity|Electricity|Wind"
    mewk_w["Unit"] = "GW"        

    mewk_wof = pd.DataFrame.from_dict(mewk_wof, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_wof["Variable"] = "Capacity|Electricity|Wind|Offshore"
    mewk_wof["Unit"] = "GW"            
    
    mewk_won = pd.DataFrame.from_dict(mewk_won, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_won["Variable"] = "Capacity|Electricity|Wind|Onshore"
    mewk_won["Unit"] = "GW"           
    
    mewk_geo = pd.DataFrame.from_dict(mewk_geo, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_geo["Variable"] = "Capacity|Electricity|Geothermal"
    mewk_geo["Unit"] = "GW"
    
    rery_tot = pd.DataFrame.from_dict(rery_tot, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_tot["Variable"] = "Primary Energy"
    rery_tot["Unit"] = "EJ/yr"    
    
    rery_c = pd.DataFrame.from_dict(rery_c, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_c["Variable"] = "Primary Energy|Coal"
    rery_c["Unit"] = "EJ/yr"     

    rery_g = pd.DataFrame.from_dict(rery_g, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_g["Variable"] = "Primary Energy|Gas"
    rery_g["Unit"] = "EJ/yr"    
    
    rery_o = pd.DataFrame.from_dict(rery_o, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_o["Variable"] = "Primary Energy|Oil"
    rery_o["Unit"] = "EJ/yr"
    
    rery_b = pd.DataFrame.from_dict(rery_b, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_b["Variable"] = "Primary Energy|Biomass"
    rery_b["Unit"] = "EJ/yr"
    
    rery_s = pd.DataFrame.from_dict(rery_s, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_s["Variable"] = "Primary Energy|Solar"
    rery_s["Unit"] = "EJ/yr"
    
    rery_w = pd.DataFrame.from_dict(rery_w, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_w["Variable"] = "Primary Energy|Wind"
    rery_w["Unit"] = "EJ/yr"
    
    rery_geo = pd.DataFrame.from_dict(rery_geo, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_geo["Variable"] = "Primary Energy|Geothermal"
    rery_geo["Unit"] = "EJ/yr"    
    
    rery_h = pd.DataFrame.from_dict(rery_h, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_h["Variable"] = "Primary Energy|Hydro"
    rery_h["Unit"] = "EJ/yr" 
    
    rery_n = pd.DataFrame.from_dict(rery_n, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_n["Variable"] = "Primary Energy|Nuclear"
    rery_n["Unit"] = "EJ/yr" 
    
    fret = pd.DataFrame.from_dict(fret, orient='index').reset_index().rename(columns={"index":"Region"})
    fret["Variable"] = "Secondary Energy|Electricity"
    fret["Unit"] = "EJ/yr"
    
    # Merge variables
    scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
                            mewk_btot, mewk_cccs, mewk_c, mewk_ctot, mewk_gccs, mewk_g, mewk_gtot, mewk_hy, mewk_n,
                            mewk_o, mewk_scsp, mewk_spv, mewk_stot, mewk_w, mewk_wof, mewk_won, mewk_geo, rery_tot,
                           rery_c, rery_g, rery_o, rery_b, rery_s, rery_w, rery_geo, rery_h, rery_n, fret])
    
    # Add model version here
    scenario["Model"] = "E3ME v1.0"
    # Add scenario name
    scenario["Scenario"] = str(m)
    # Correct scenario name
    scenario["Scenario"] = np.where(scenario["Scenario"]=="SUP_1p5C_Dif10", "SUP_1p5C_Regional", scenario["Scenario"])
    scenario["Scenario"] = np.where(scenario["Scenario"]=="SUP_2C_Dif10", "SUP_2C_Regional", scenario["Scenario"])
    # Calculate world region
    world = scenario.groupby(["Model", "Scenario", "Variable", "Unit"]).sum().reset_index()
    world["Region"] = "World"
    # Get rid of carbon price
    world = world.loc[world["Variable"]!="Price|Carbon"]
    scenario = scenario.append(world)
    # Reorder cols and format to 5 year increments
    scenario = scenario.filter(["Model", "Scenario", "Region", "Variable", "Unit", 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050])
    # Append to main results df
    results = results.append(scenario)
    
scenario

Processing: SUP_NPi_Default


C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:327: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:344: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:348: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


Processing: SUP_1p5C_Default


C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:327: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:344: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:348: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


Processing: SUP_2C_Default


...

Processing: SUP_1p5C_Dif10


C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:327: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:344: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:348: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


Processing: SUP_2C_Dif10


C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:327: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:344: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_8324\3723269502.py:348: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


,Model,Scenario,Region,Variable,Unit,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,E3ME v1.0,SUP_2C_Regional,Belgium,GDP|MER,billion US$2010/yr,482.976325,515.173422,510.728811,597.421910,625.325892,644.890525,675.863772,718.856355,757.338159
1,E3ME v1.0,SUP_2C_Regional,Denmark,GDP|MER,billion US$2010/yr,323.410136,344.802749,367.907135,428.612907,477.536001,514.073179,553.893296,599.850241,649.580146
2,E3ME v1.0,SUP_2C_Regional,Germany,GDP|MER,billion US$2010/yr,3410.652000,3717.737363,3762.749218,4281.433590,4551.464820,4665.605420,4838.677988,5199.406240,5538.031555
3,E3ME v1.0,SUP_2C_Regional,Greece,GDP|MER,billion US$2010/yr,300.621874,245.748796,232.231051,285.455199,282.345991,290.997225,302.458293,330.056790,358.552871
4,E3ME v1.0,SUP_2C_Regional,Spain,GDP|MER,billion US$2010/yr,1426.703136,1424.853106,1390.851656,1718.658419,1852.841786,1981.749376,2130.853515,2272.419214,2412.774114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,E3ME v1.0,SUP_2C_Regional,World,Primary Energy|Oil,EJ/yr,0.000000,0.000000,163.321564,179.544070,178.937358,173.954278,169.387772,164.375735,156.059166
39,E3ME v1.0,SUP_2C_Regional,World,Primary Energy|Solar,EJ/yr,0.144651,1.077233,4.109138,11.217998,22.120293,37.356348,54.077180,78.871784,106.108653
40,E3ME v1.0,SUP_2C_Regional,World,Primary Energy|Wind,EJ/yr,1.207848,2.953090,7.372816,13.198271,18.660097,26.091534,32.134307,33.262766,30.705964
41,E3ME v1.0,SUP_2C_Regional,World,Secondary Energy|Electricity,EJ/yr,66.625303,74.887626,84.611743,98.742678,106.963017,120.471680,135.676704,153.804463,174.884606


In [7]:
results.to_excel(r"Output/NAVIGATE/E3ME_task_carbonprice_230623.xlsx")

In [22]:
# SECTORAL DISAGGREGATION

# GDP (bn $ 2014) -> RGDP 
# Employment ( mn persons) -> REMP
# Investment -> RSK
# Public Consumption -> RSG
# Private Consumption -> RSC
# Output -> QR
# Exports -> QRX
# Imports -> QRM

eu = ['Belgium', 'Denmark', 'Germany', 'Greece', 'Spain', 'France', 'Ireland', 'Italy', 'Luxembourg', 'Netherlands',
       'Austria', 'Portugal', 'Finland', 'Sweden', 'Czech Republic', 'Estonia', 'Cyprus', 'Latvia', 'Lithuania', 
       'Hungary', 'Malta', 'Poland', 'Slovenia', 'Slovakia', 'Bulgaria', 'Romania', 'Croatia', "UK"]
rep = ["Egypt", "Rest of Central Africa OPEC", "Rest of North Africa", "Rest of North Africa OPEC", "Rest of OPEC","UAE"]
cty = ["EU28", "China", "India", "USA", "Canada", "Russia", "Korea", "Japan", "Saudi Arabia", "Brazil"]

# Define sectors
agr = [0,42]
ff = [1,2,8]
elc = [21]
eii = [*range(4,8), *range(9,14)]
oth = [*range(14,21)]
con = [24]
ser = [3,22,23, *range(25,42), 43]

# EU sectors
agr_eu = [*range(0, 3)]
ff_eu = [3,4,11]
elc_eu = [25]
eii_eu = [*range(6,11), *range(12,17)]
oth_eu = [*range(17,25)]
con_eu = [29]
ser_eu = [5, *range(26,29), *range(30,70)]


mre_list = ["SUP_NPi_Default", "SUP_2C_Default", "SUP_1p5C_Default",
           "SUP_2C_Lab", "SUP_1p5C_Lab", "SUP_2C_Lump", "SUP_1p5C_Lump",
           "SUP_2C_VAT", "SUP_1p5C_VAT"]


# mre_list = ["SUP_NPi_Default", "SUP_1p5C_Default", "SUP_1p5C_Lab", "SUP_1p5C_Lump","SUP_1p5C_VAT"]

years = range(2010, 2051)
results_pt2 = pd.DataFrame()

for m in mre_list:
    # Load mre
    print("Processing: " + str(m))
    mre = MRE("C:/E3ME/EEIST_NAVIGATE/Output/" + str(m) + ".mre")
    
    # 2d variables
    # Aggregate GDP (+ conversion)
    rgdp = pd.DataFrame((mre['RGDP'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rgdp["Variable"] = "1 GDP"
    rgdp["Unit"] = "billion US$2010/yr"
    
    # Aggregate employment (+ conversion)
    remp = pd.DataFrame((mre['REMP'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    remp["Variable"] = "2 Employment"
    remp["Unit"] = "million"  
    
    # Aggregate investment (+ conversion)
    rsk = pd.DataFrame((mre['RSK'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsk["Variable"] = "3 Investment"
    rsk["Unit"] = "billion US$2010/yr"
    
    # Aggregate gov consumption (+ conversion)
    rsg = pd.DataFrame((mre['RSG'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsg["Variable"] = "4 Public Consumption"
    rsg["Unit"] = "billion US$2010/yr"
    
    # Aggregate household consumption (+ conversion)
    rsc = pd.DataFrame((mre['RSC'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsc["Variable"] = "5 Private Consumption"
    rsc["Unit"] = "billion US$2010/yr"
    
    # Aggregate output
    qr = {}
    qr_tot = {}
    for r, reg in enumerate(regions):
        # qr (output)
        qr[reg] = pd.DataFrame(mre["QR"][r], columns=years)
        
        qr_tot[reg] = (qr[reg].sum()/1000)*1.33
    
    qr_tot = pd.DataFrame.from_dict(qr_tot, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_tot["Variable"] = "6 Output"
    qr_tot["Unit"] = "billion US$2010/yr"
    
    # Aggregate export (+ conversion)
    rsx = pd.DataFrame((mre['RSX'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsx["Variable"] = "7 Export"
    rsx["Unit"] = "billion US$2010/yr"
    
    # Aggregate import (+ conversion)
    rsm = pd.DataFrame((mre['RSM'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsm["Variable"] = "8 Import"
    rsm["Unit"] = "billion US$2010/yr"
    
    # 3D Vars
    qr_agr = {}
    qr_ff = {}
    qr_elc = {}
    qr_eii = {}
    qr_oth = {}
    qr_con = {}
    qr_ser = {}
    
    yre = {}
    yre_tot = {}
    yre_agr = {}
    yre_ff = {}
    yre_elc = {}
    yre_eii = {}
    yre_oth = {}
    yre_con = {}
    yre_ser = {}
    
    qex = {}
    qex_tot = {}
    qex_agr = {}
    qex_ff = {}
    qex_elc = {}
    qex_eii = {}
    qex_oth = {}
    qex_con = {}
    qex_ser = {}
    
    qem = {}
    qem_tot = {}
    qem_agr = {}
    qem_ff = {}
    qem_elc = {}
    qem_eii = {}
    qem_oth = {}
    qem_con = {}
    qem_ser = {}

    for r, reg in enumerate(regions):
        if reg in eu:
            # if region in eu, do this part seperate
            # qr (output) (+ Conversion)       
            qr_agr[reg] = (qr[reg].iloc[agr_eu].sum()/1000)*1.33
            qr_ff[reg] = (qr[reg].iloc[ff_eu].sum()/1000)*1.33
            qr_elc[reg] = (qr[reg].iloc[elc_eu].sum()/1000)*1.33
            qr_eii[reg] = (qr[reg].iloc[eii_eu].sum()/1000)*1.33
            qr_oth[reg] = (qr[reg].iloc[oth_eu].sum()/1000)*1.33
            qr_con[reg] = (qr[reg].iloc[con_eu].sum()/1000)*1.33
            qr_ser[reg] = (qr[reg].iloc[ser_eu].sum()/1000)*1.33

            # yre (employment) (+ Conversion)
            yre[reg] = pd.DataFrame(mre["YRE"][r], columns=years)
            yre_agr[reg] = (yre[reg].iloc[agr_eu].sum()/1000)
            yre_ff[reg] = (yre[reg].iloc[ff_eu].sum()/1000)
            yre_elc[reg] = (yre[reg].iloc[elc_eu].sum()/1000)
            yre_eii[reg] = (yre[reg].iloc[eii_eu].sum()/1000)
            yre_oth[reg] = (yre[reg].iloc[oth_eu].sum()/1000)
            yre_con[reg] = (yre[reg].iloc[con_eu].sum()/1000)
            yre_ser[reg] = (yre[reg].iloc[ser_eu].sum()/1000)

            # qex (exports) (+ Conversion)
            qex[reg] = pd.DataFrame(mre["QEX"][r], columns=years)
            qex_agr[reg] = (qex[reg].iloc[agr_eu].sum()/1000)*1.33
            qex_ff[reg] = (qex[reg].iloc[ff_eu].sum()/1000)*1.33
            qex_elc[reg] = (qex[reg].iloc[elc_eu].sum()/1000)*1.33
            qex_eii[reg] = (qex[reg].iloc[eii_eu].sum()/1000)*1.33
            qex_oth[reg] = (qex[reg].iloc[oth_eu].sum()/1000)*1.33
            qex_con[reg] = (qex[reg].iloc[con_eu].sum()/1000)*1.33
            qex_ser[reg] = (qex[reg].iloc[ser_eu].sum()/1000)*1.33

            # qem (imports) (+ Conversion)
            qem[reg] = pd.DataFrame(mre["QEM"][r], columns=years)
            qem_agr[reg] = (qem[reg].iloc[agr_eu].sum()/1000)*1.33
            qem_ff[reg] = (qem[reg].iloc[ff_eu].sum()/1000)*1.33
            qem_elc[reg] = (qem[reg].iloc[elc_eu].sum()/1000)*1.33
            qem_eii[reg] = (qem[reg].iloc[eii_eu].sum()/1000)*1.33
            qem_oth[reg] = (qem[reg].iloc[oth_eu].sum()/1000)*1.33
            qem_con[reg] = (qem[reg].iloc[con_eu].sum()/1000)*1.33
            qem_ser[reg] = (qem[reg].iloc[ser_eu].sum()/1000)*1.33
    
        else:
            # qr (output) (+ Conversion)       
            qr_agr[reg] = (qr[reg].iloc[agr].sum()/1000)*1.33
            qr_ff[reg] = (qr[reg].iloc[ff].sum()/1000)*1.33
            qr_elc[reg] = (qr[reg].iloc[elc].sum()/1000)*1.33
            qr_eii[reg] = (qr[reg].iloc[eii].sum()/1000)*1.33
            qr_oth[reg] = (qr[reg].iloc[oth].sum()/1000)*1.33
            qr_con[reg] = (qr[reg].iloc[con].sum()/1000)*1.33
            qr_ser[reg] = (qr[reg].iloc[ser].sum()/1000)*1.33

            # yre (employment) (+ Conversion)
            yre[reg] = pd.DataFrame(mre["YRE"][r], columns=years)
            yre_agr[reg] = (yre[reg].iloc[agr].sum()/1000)
            yre_ff[reg] = (yre[reg].iloc[ff].sum()/1000)
            yre_elc[reg] = (yre[reg].iloc[elc].sum()/1000)
            yre_eii[reg] = (yre[reg].iloc[eii].sum()/1000)
            yre_oth[reg] = (yre[reg].iloc[oth].sum()/1000)
            yre_con[reg] = (yre[reg].iloc[con].sum()/1000)
            yre_ser[reg] = (yre[reg].iloc[ser].sum()/1000)

            # qex (exports) (+ Conversion)
            qex[reg] = pd.DataFrame(mre["QEX"][r], columns=years)
            qex_agr[reg] = (qex[reg].iloc[agr].sum()/1000)*1.33
            qex_ff[reg] = (qex[reg].iloc[ff].sum()/1000)*1.33
            qex_elc[reg] = (qex[reg].iloc[elc].sum()/1000)*1.33
            qex_eii[reg] = (qex[reg].iloc[eii].sum()/1000)*1.33
            qex_oth[reg] = (qex[reg].iloc[oth].sum()/1000)*1.33
            qex_con[reg] = (qex[reg].iloc[con].sum()/1000)*1.33
            qex_ser[reg] = (qex[reg].iloc[ser].sum()/1000)*1.33

            # qem (imports) (+ Conversion)
            qem[reg] = pd.DataFrame(mre["QEM"][r], columns=years)
            qem_agr[reg] = (qem[reg].iloc[agr].sum()/1000)*1.33
            qem_ff[reg] = (qem[reg].iloc[ff].sum()/1000)*1.33
            qem_elc[reg] = (qem[reg].iloc[elc].sum()/1000)*1.33
            qem_eii[reg] = (qem[reg].iloc[eii].sum()/1000)*1.33
            qem_oth[reg] = (qem[reg].iloc[oth].sum()/1000)*1.33
            qem_con[reg] = (qem[reg].iloc[con].sum()/1000)*1.33
            qem_ser[reg] = (qem[reg].iloc[ser].sum()/1000)*1.33
    
    
#     qr = pd.DataFrame.from_dict(qr, orient='index').reset_index().rename(columns={"index":"Region"})
#     qr["Variable"] = "Aggregate Output"
#     qr["Unit"] = "billion US$2010/yr" 
    
    # Labels Output   
    qr_agr = pd.DataFrame.from_dict(qr_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_agr["Variable"] = "1 Output|Agriculture"
    qr_agr["Unit"] = "billion US$2010/yr" 

    qr_ff = pd.DataFrame.from_dict(qr_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_ff["Variable"] = "2 Output|Fossil fuels"
    qr_ff["Unit"] = "billion US$2010/yr" 

    qr_elc = pd.DataFrame.from_dict(qr_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_elc["Variable"] = "3 Output|Electricity"
    qr_elc["Unit"] = "billion US$2010/yr"     

    qr_eii = pd.DataFrame.from_dict(qr_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_eii["Variable"] = "4 Output|Energy-intensive industries"
    qr_eii["Unit"] = "billion US$2010/yr"     

    qr_oth = pd.DataFrame.from_dict(qr_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_oth["Variable"] = "5 Output|Other industries"
    qr_oth["Unit"] = "billion US$2010/yr"     

    qr_con = pd.DataFrame.from_dict(qr_con, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_con["Variable"] = "6 Output|Construction"
    qr_con["Unit"] = "billion US$2010/yr" 
    
    qr_ser = pd.DataFrame.from_dict(qr_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_ser["Variable"] = "7 Output|Services"
    qr_ser["Unit"] = "billion US$2010/yr"     
    
    # Labels Employment
    yre_agr = pd.DataFrame.from_dict(yre_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_agr["Variable"] = "1 Employment|Agriculture"
    yre_agr["Unit"] = "millions" 

    yre_ff = pd.DataFrame.from_dict(yre_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_ff["Variable"] = "2 Employment|Fossil fuels"
    yre_ff["Unit"] = "millions" 

    yre_elc = pd.DataFrame.from_dict(yre_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_elc["Variable"] = "3 Employment|Electricity"
    yre_elc["Unit"] = "millions"   

    yre_eii = pd.DataFrame.from_dict(yre_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_eii["Variable"] = "4 Employment|Energy-intensive industries"
    yre_eii["Unit"] = "millions"     

    yre_oth = pd.DataFrame.from_dict(yre_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_oth["Variable"] = "5 Employment|Other industries"
    yre_oth["Unit"] = "millions"     

    yre_con = pd.DataFrame.from_dict(yre_con, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_con["Variable"] = "6 Employment|Construction"
    yre_con["Unit"] = "millions" 
    
    yre_ser = pd.DataFrame.from_dict(yre_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_ser["Variable"] = "7 Employment|Services"
    yre_ser["Unit"] = "millions"   
    
    
    # Labels Exports   
    qex_agr = pd.DataFrame.from_dict(qex_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_agr["Variable"] = "1 Exports|Agriculture"
    qex_agr["Unit"] = "billion US$2010/yr" 

    qex_ff = pd.DataFrame.from_dict(qex_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_ff["Variable"] = "2 Exports|Fossil fuels"
    qex_ff["Unit"] = "billion US$2010/yr" 

    qex_elc = pd.DataFrame.from_dict(qex_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_elc["Variable"] = "3 Exports|Electricity"
    qex_elc["Unit"] = "billion US$2010/yr"     

    qex_eii = pd.DataFrame.from_dict(qex_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_eii["Variable"] = "4 Exports|Energy-intensive industries"
    qex_eii["Unit"] = "billion US$2010/yr"     

    qex_oth = pd.DataFrame.from_dict(qex_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_oth["Variable"] = "5 Exports|Other industries"
    qex_oth["Unit"] = "billion US$2010/yr"     

    qex_con = pd.DataFrame.from_dict(qex_con, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_con["Variable"] = "6 Exports|Construction"
    qex_con["Unit"] = "billion US$2010/yr" 
    
    qex_ser = pd.DataFrame.from_dict(qex_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_ser["Variable"] = "7 Exports|Services"
    qex_ser["Unit"] = "billion US$2010/yr"  
    
    
    # Labels Imports   
    qem_agr = pd.DataFrame.from_dict(qem_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_agr["Variable"] = "1 Imports|Agriculture"
    qem_agr["Unit"] = "billion US$2010/yr" 

    qem_ff = pd.DataFrame.from_dict(qem_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_ff["Variable"] = "2 Imports|Fossil fuels"
    qem_ff["Unit"] = "billion US$2010/yr" 

    qem_elc = pd.DataFrame.from_dict(qem_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_elc["Variable"] = "3 Imports|Electricity"
    qem_elc["Unit"] = "billion US$2010/yr"     

    qem_eii = pd.DataFrame.from_dict(qem_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_eii["Variable"] = "4 Imports|Energy-intensive industries"
    qem_eii["Unit"] = "billion US$2010/yr"     

    qem_oth = pd.DataFrame.from_dict(qem_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_oth["Variable"] = "5 Imports|Other industries"
    qem_oth["Unit"] = "billion US$2010/yr"     

    qem_con = pd.DataFrame.from_dict(qem_con, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_con["Variable"] = "6 Imports|Construction"
    qem_con["Unit"] = "billion US$2010/yr" 
    
    qem_ser = pd.DataFrame.from_dict(qem_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_ser["Variable"] = "7 Imports|Services"
    qem_ser["Unit"] = "billion US$2010/yr" 
 
    # Merge variables
    scenario = rgdp.append([remp, rsk, rsg, rsc, qr_tot, rsx, rsm, 
                            qr_agr, qr_ff, qr_elc, qr_eii, qr_oth, qr_con, qr_ser,
                           yre_agr, yre_ff, yre_elc, yre_eii, yre_oth, yre_con, yre_ser,
                           qex_agr, qex_ff, qex_elc, qex_eii, qex_oth, qex_con, qex_ser,
                           qem_agr, qem_ff, qem_elc, qem_eii, qem_oth, qem_con, qem_ser])
    
    # Add model version here
    scenario["Model"] = "E3ME_v1.0"
    # Add scenario name
    scenario["Scenario"] = str(m)
    
    # Calculate world region
    world = scenario.groupby(["Model", "Scenario", "Variable", "Unit"]).sum().reset_index()
    world["Region"] = "World"
    scenario = scenario.append(world)
    
    # Calculate EU
    scenario["Region"] = np.where(scenario["Region"].isin(eu), "EU28", scenario["Region"])
    
    # Calculate REP
    scenario["Region"] = np.where(scenario["Region"].isin(rep), "REP", scenario["Region"])
    scenario = scenario.groupby(["Model", "Scenario", "Variable", "Unit", "Region"]).sum().reset_index()

    
#     # Reorder cols and format to 5 year increments
    scenario = scenario.filter(["Model", "Scenario", "Region", "Variable", "Unit", 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050])
    # Append to main results df
    results_pt2 = results_pt2.append(scenario)

scenario

Processing: SUP_NPi_Default


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\3859236985.py:346: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([remp, rsk, rsg, rsc, qr_tot, rsx, rsm,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\3859236985.py:360: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\3859236985.py:373: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_pt2 = results_pt2.append(scenario)


Processing: SUP_2C_Default


KeyboardInterrupt: 

In [24]:
# SECTORAL DISAGGREGATION - exactly the same as above, just no numbers

# GDP (bn $ 2014) -> RGDP 
# Employment ( mn persons) -> REMP
# Investment -> RSK
# Public Consumption -> RSG
# Private Consumption -> RSC
# Output -> QR
# Exports -> QRX
# Imports -> QRM

eu = ['Belgium', 'Denmark', 'Germany', 'Greece', 'Spain', 'France', 'Ireland', 'Italy', 'Luxembourg', 'Netherlands',
       'Austria', 'Portugal', 'Finland', 'Sweden', 'Czech Republic', 'Estonia', 'Cyprus', 'Latvia', 'Lithuania', 
       'Hungary', 'Malta', 'Poland', 'Slovenia', 'Slovakia', 'Bulgaria', 'Romania', 'Croatia', "UK"]
rep = ["Egypt", "Rest of Central Africa OPEC", "Rest of North Africa", "Rest of North Africa OPEC", "Rest of OPEC","UAE"]
cty = ["EU28", "China", "India", "USA", "Canada", "Russia", "Korea", "Japan", "Saudi Arabia", "Brazil"]

# Define sectors
agr = [0,42]
ff = [1,2,8]
elc = [21]
eii = [*range(4,8), *range(9,14)]
oth = [*range(14,21)]
con = [24]
ser = [3,22,23, *range(25,42), 43]

# EU sectors
agr_eu = [*range(0, 3)]
ff_eu = [3,4,11]
elc_eu = [25]
eii_eu = [*range(6,11), *range(12,17)]
oth_eu = [*range(17,25)]
con_eu = [29]
ser_eu = [5, *range(26,29), *range(30,70)]


mre_list = ["SUP_NPi_Default", "SUP_2C_Default", "SUP_1p5C_Default",
           "SUP_2C_Lab", "SUP_1p5C_Lab", "SUP_2C_Lump", "SUP_1p5C_Lump",
           "SUP_2C_VAT", "SUP_1p5C_VAT"]


# mre_list = ["SUP_NPi_Default", "SUP_1p5C_Default", "SUP_1p5C_Lab", "SUP_1p5C_Lump","SUP_1p5C_VAT"]

years = range(2010, 2051)
results_pt2 = pd.DataFrame()

for m in mre_list:
    # Load mre
    print("Processing: " + str(m))
    mre = MRE("C:/E3ME/EEIST_NAVIGATE/Output/" + str(m) + ".mre")
    
    # 2d variables
    # Aggregate GDP (+ conversion)
    rgdp = pd.DataFrame((mre['RGDP'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rgdp["Variable"] = "GDP (bn $ 2014)"
    rgdp["Unit"] = "billion US$2010/yr"
    rgdp["Category"] = "Aggregate"
    
    
    # Aggregate employment (+ conversion)
    remp = pd.DataFrame((mre['REMP'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    remp["Variable"] = "Employment ( mn persons)"
    remp["Unit"] = "million"  
    remp["Category"] = "Aggregate"
    
    # Aggregate investment (+ conversion)
    rsk = pd.DataFrame((mre['RSK'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsk["Variable"] = "Investment"
    rsk["Unit"] = "billion US$2010/yr"
    rsk["Category"] = "Aggregate"
    
    # Aggregate gov consumption (+ conversion)
    rsg = pd.DataFrame((mre['RSG'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsg["Variable"] = "Public Consumption"
    rsg["Unit"] = "billion US$2010/yr"
    rsg["Category"] = "Aggregate"
    
    # Aggregate household consumption (+ conversion)
    rsc = pd.DataFrame((mre['RSC'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsc["Variable"] = "Private Consumption"
    rsc["Unit"] = "billion US$2010/yr"
    rsc["Category"] = "Aggregate"
    
    # Aggregate output
    qr = {}
    qr_tot = {}
    for r, reg in enumerate(regions):
        # qr (output)
        qr[reg] = pd.DataFrame(mre["QR"][r], columns=years)
        
        qr_tot[reg] = (qr[reg].sum()/1000)*1.33
    
    qr_tot = pd.DataFrame.from_dict(qr_tot, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_tot["Variable"] = "Output"
    qr_tot["Unit"] = "billion US$2010/yr"
    qr_tot["Category"] = "Aggregate"
    
    # Aggregate export (+ conversion)
    rsx = pd.DataFrame((mre['RSX'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsx["Variable"] = "Exports"
    rsx["Unit"] = "billion US$2010/yr"
    rsx["Category"] = "Aggregate"
    
    # Aggregate import (+ conversion)
    rsm = pd.DataFrame((mre['RSM'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsm["Variable"] = "Imports"
    rsm["Unit"] = "billion US$2010/yr"
    rsm["Category"] = "Aggregate"
    
    # 3D Vars
    qr_agr = {}
    qr_ff = {}
    qr_elc = {}
    qr_eii = {}
    qr_oth = {}
    qr_con = {}
    qr_ser = {}
    
    yre = {}
    yre_tot = {}
    yre_agr = {}
    yre_ff = {}
    yre_elc = {}
    yre_eii = {}
    yre_oth = {}
    yre_con = {}
    yre_ser = {}
    
    qex = {}
    qex_tot = {}
    qex_agr = {}
    qex_ff = {}
    qex_elc = {}
    qex_eii = {}
    qex_oth = {}
    qex_con = {}
    qex_ser = {}
    
    qem = {}
    qem_tot = {}
    qem_agr = {}
    qem_ff = {}
    qem_elc = {}
    qem_eii = {}
    qem_oth = {}
    qem_con = {}
    qem_ser = {}

    for r, reg in enumerate(regions):
        if reg in eu:
            # if region in eu, do this part seperate
            # qr (output) (+ Conversion)       
            qr_agr[reg] = (qr[reg].iloc[agr_eu].sum()/1000)*1.33
            qr_ff[reg] = (qr[reg].iloc[ff_eu].sum()/1000)*1.33
            qr_elc[reg] = (qr[reg].iloc[elc_eu].sum()/1000)*1.33
            qr_eii[reg] = (qr[reg].iloc[eii_eu].sum()/1000)*1.33
            qr_oth[reg] = (qr[reg].iloc[oth_eu].sum()/1000)*1.33
            qr_con[reg] = (qr[reg].iloc[con_eu].sum()/1000)*1.33
            qr_ser[reg] = (qr[reg].iloc[ser_eu].sum()/1000)*1.33

            # yre (employment) (+ Conversion)
            yre[reg] = pd.DataFrame(mre["YRE"][r], columns=years)
            yre_agr[reg] = (yre[reg].iloc[agr_eu].sum()/1000)
            yre_ff[reg] = (yre[reg].iloc[ff_eu].sum()/1000)
            yre_elc[reg] = (yre[reg].iloc[elc_eu].sum()/1000)
            yre_eii[reg] = (yre[reg].iloc[eii_eu].sum()/1000)
            yre_oth[reg] = (yre[reg].iloc[oth_eu].sum()/1000)
            yre_con[reg] = (yre[reg].iloc[con_eu].sum()/1000)
            yre_ser[reg] = (yre[reg].iloc[ser_eu].sum()/1000)

            # qex (exports) (+ Conversion)
            qex[reg] = pd.DataFrame(mre["QEX"][r], columns=years)
            qex_agr[reg] = (qex[reg].iloc[agr_eu].sum()/1000)*1.33
            qex_ff[reg] = (qex[reg].iloc[ff_eu].sum()/1000)*1.33
            qex_elc[reg] = (qex[reg].iloc[elc_eu].sum()/1000)*1.33
            qex_eii[reg] = (qex[reg].iloc[eii_eu].sum()/1000)*1.33
            qex_oth[reg] = (qex[reg].iloc[oth_eu].sum()/1000)*1.33
            qex_con[reg] = (qex[reg].iloc[con_eu].sum()/1000)*1.33
            qex_ser[reg] = (qex[reg].iloc[ser_eu].sum()/1000)*1.33

            # qem (imports) (+ Conversion)
            qem[reg] = pd.DataFrame(mre["QEM"][r], columns=years)
            qem_agr[reg] = (qem[reg].iloc[agr_eu].sum()/1000)*1.33
            qem_ff[reg] = (qem[reg].iloc[ff_eu].sum()/1000)*1.33
            qem_elc[reg] = (qem[reg].iloc[elc_eu].sum()/1000)*1.33
            qem_eii[reg] = (qem[reg].iloc[eii_eu].sum()/1000)*1.33
            qem_oth[reg] = (qem[reg].iloc[oth_eu].sum()/1000)*1.33
            qem_con[reg] = (qem[reg].iloc[con_eu].sum()/1000)*1.33
            qem_ser[reg] = (qem[reg].iloc[ser_eu].sum()/1000)*1.33
    
        else:
            # qr (output) (+ Conversion)       
            qr_agr[reg] = (qr[reg].iloc[agr].sum()/1000)*1.33
            qr_ff[reg] = (qr[reg].iloc[ff].sum()/1000)*1.33
            qr_elc[reg] = (qr[reg].iloc[elc].sum()/1000)*1.33
            qr_eii[reg] = (qr[reg].iloc[eii].sum()/1000)*1.33
            qr_oth[reg] = (qr[reg].iloc[oth].sum()/1000)*1.33
            qr_con[reg] = (qr[reg].iloc[con].sum()/1000)*1.33
            qr_ser[reg] = (qr[reg].iloc[ser].sum()/1000)*1.33

            # yre (employment) (+ Conversion)
            yre[reg] = pd.DataFrame(mre["YRE"][r], columns=years)
            yre_agr[reg] = (yre[reg].iloc[agr].sum()/1000)
            yre_ff[reg] = (yre[reg].iloc[ff].sum()/1000)
            yre_elc[reg] = (yre[reg].iloc[elc].sum()/1000)
            yre_eii[reg] = (yre[reg].iloc[eii].sum()/1000)
            yre_oth[reg] = (yre[reg].iloc[oth].sum()/1000)
            yre_con[reg] = (yre[reg].iloc[con].sum()/1000)
            yre_ser[reg] = (yre[reg].iloc[ser].sum()/1000)

            # qex (exports) (+ Conversion)
            qex[reg] = pd.DataFrame(mre["QEX"][r], columns=years)
            qex_agr[reg] = (qex[reg].iloc[agr].sum()/1000)*1.33
            qex_ff[reg] = (qex[reg].iloc[ff].sum()/1000)*1.33
            qex_elc[reg] = (qex[reg].iloc[elc].sum()/1000)*1.33
            qex_eii[reg] = (qex[reg].iloc[eii].sum()/1000)*1.33
            qex_oth[reg] = (qex[reg].iloc[oth].sum()/1000)*1.33
            qex_con[reg] = (qex[reg].iloc[con].sum()/1000)*1.33
            qex_ser[reg] = (qex[reg].iloc[ser].sum()/1000)*1.33

            # qem (imports) (+ Conversion)
            qem[reg] = pd.DataFrame(mre["QEM"][r], columns=years)
            qem_agr[reg] = (qem[reg].iloc[agr].sum()/1000)*1.33
            qem_ff[reg] = (qem[reg].iloc[ff].sum()/1000)*1.33
            qem_elc[reg] = (qem[reg].iloc[elc].sum()/1000)*1.33
            qem_eii[reg] = (qem[reg].iloc[eii].sum()/1000)*1.33
            qem_oth[reg] = (qem[reg].iloc[oth].sum()/1000)*1.33
            qem_con[reg] = (qem[reg].iloc[con].sum()/1000)*1.33
            qem_ser[reg] = (qem[reg].iloc[ser].sum()/1000)*1.33
    
    
#     qr = pd.DataFrame.from_dict(qr, orient='index').reset_index().rename(columns={"index":"Region"})
#     qr["Variable"] = "Aggregate Output"
#     qr["Unit"] = "billion US$2010/yr" 
    
    # Labels Output   
    qr_agr = pd.DataFrame.from_dict(qr_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_agr["Variable"] = "Agriculture"
    qr_agr["Unit"] = "billion US$2010/yr"
    qr_agr["Category"] = "Production by sector"

    qr_ff = pd.DataFrame.from_dict(qr_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_ff["Variable"] = "Fossil fuels"
    qr_ff["Unit"] = "billion US$2010/yr" 
    qr_ff["Category"] = "Production by sector"

    qr_elc = pd.DataFrame.from_dict(qr_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_elc["Variable"] = "Electricity"
    qr_elc["Unit"] = "billion US$2010/yr"     
    qr_elc["Category"] = "Production by sector"

    qr_eii = pd.DataFrame.from_dict(qr_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_eii["Variable"] = "Energy-intensive industries"
    qr_eii["Unit"] = "billion US$2010/yr"     
    qr_eii["Category"] = "Production by sector"

    qr_oth = pd.DataFrame.from_dict(qr_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_oth["Variable"] = "Other industries"
    qr_oth["Unit"] = "billion US$2010/yr"     
    qr_oth["Category"] = "Production by sector"

    qr_con = pd.DataFrame.from_dict(qr_con, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_con["Variable"] = "Construction"
    qr_con["Unit"] = "billion US$2010/yr" 
    qr_con["Category"] = "Production by sector"
    
    qr_ser = pd.DataFrame.from_dict(qr_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    qr_ser["Variable"] = "Services"
    qr_ser["Unit"] = "billion US$2010/yr" 
    qr_ser["Category"] = "Production by sector"
    
    # Labels Employment
    yre_agr = pd.DataFrame.from_dict(yre_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_agr["Variable"] = "Agriculture"
    yre_agr["Unit"] = "millions" 
    yre_agr["Category"] = "Employment by sector"

    yre_ff = pd.DataFrame.from_dict(yre_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_ff["Variable"] = "Fossil fuels"
    yre_ff["Unit"] = "millions" 
    yre_ff["Category"] = "Employment by sector"

    yre_elc = pd.DataFrame.from_dict(yre_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_elc["Variable"] = "Electricity"
    yre_elc["Unit"] = "millions"   
    yre_elc["Category"] = "Employment by sector"

    yre_eii = pd.DataFrame.from_dict(yre_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_eii["Variable"] = "Energy-intensive industries"
    yre_eii["Unit"] = "millions"  
    yre_eii["Category"] = "Employment by sector"

    yre_oth = pd.DataFrame.from_dict(yre_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_oth["Variable"] = "Other industries"
    yre_oth["Unit"] = "millions"     
    yre_oth["Category"] = "Employment by sector"

    yre_con = pd.DataFrame.from_dict(yre_con, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_con["Variable"] = "Construction"
    yre_con["Unit"] = "millions" 
    yre_con["Category"] = "Employment by sector"
    
    yre_ser = pd.DataFrame.from_dict(yre_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    yre_ser["Variable"] = "Services"
    yre_ser["Unit"] = "millions"   
    yre_ser["Category"] = "Employment by sector"
    
    
    # Labels Exports   
    qex_agr = pd.DataFrame.from_dict(qex_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_agr["Variable"] = "Agriculture"
    qex_agr["Unit"] = "billion US$2010/yr" 
    qex_agr["Category"] = "Exports by sector"

    qex_ff = pd.DataFrame.from_dict(qex_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_ff["Variable"] = "Fossil fuels"
    qex_ff["Unit"] = "billion US$2010/yr" 
    qex_ff["Category"] = "Exports by sector"

    qex_elc = pd.DataFrame.from_dict(qex_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_elc["Variable"] = "Electricity"
    qex_elc["Unit"] = "billion US$2010/yr"    
    qex_elc["Category"] = "Exports by sector"

    qex_eii = pd.DataFrame.from_dict(qex_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_eii["Variable"] = "Energy-intensive industries"
    qex_eii["Unit"] = "billion US$2010/yr"
    qex_eii["Category"] = "Exports by sector"

    qex_oth = pd.DataFrame.from_dict(qex_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_oth["Variable"] = "Other industries"
    qex_oth["Unit"] = "billion US$2010/yr"     
    qex_oth["Category"] = "Exports by sector"

    qex_con = pd.DataFrame.from_dict(qex_con, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_con["Variable"] = "Construction"
    qex_con["Unit"] = "billion US$2010/yr" 
    qex_con["Category"] = "Exports by sector"
    
    qex_ser = pd.DataFrame.from_dict(qex_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    qex_ser["Variable"] = "Services"
    qex_ser["Unit"] = "billion US$2010/yr"  
    qex_ser["Category"] = "Exports by sector"
    
    # Labels Imports   
    qem_agr = pd.DataFrame.from_dict(qem_agr, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_agr["Variable"] = "Agriculture"
    qem_agr["Unit"] = "billion US$2010/yr" 
    qem_agr["Category"] = "Imports by sector"

    qem_ff = pd.DataFrame.from_dict(qem_ff, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_ff["Variable"] = "Fossil fuels"
    qem_ff["Unit"] = "billion US$2010/yr" 
    qem_ff["Category"] = "Imports by sector"

    qem_elc = pd.DataFrame.from_dict(qem_elc, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_elc["Variable"] = "Electricity"
    qem_elc["Unit"] = "billion US$2010/yr"     
    qem_elc["Category"] = "Imports by sector"

    qem_eii = pd.DataFrame.from_dict(qem_eii, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_eii["Variable"] = "Energy-intensive industries"
    qem_eii["Unit"] = "billion US$2010/yr"    
    qem_eii["Category"] = "Imports by sector"

    qem_oth = pd.DataFrame.from_dict(qem_oth, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_oth["Variable"] = "Other industries"
    qem_oth["Unit"] = "billion US$2010/yr"     
    qem_oth["Category"] = "Imports by sector"

    qem_con = pd.DataFrame.from_dict(qem_con, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_con["Variable"] = "Construction"
    qem_con["Unit"] = "billion US$2010/yr" 
    qem_con["Category"] = "Imports by sector"
    
    qem_ser = pd.DataFrame.from_dict(qem_ser, orient='index').reset_index().rename(columns={"index":"Region"})
    qem_ser["Variable"] = "Services"
    qem_ser["Unit"] = "billion US$2010/yr" 
    qem_ser["Category"] = "Imports by sector"
 
    # Merge variables
    scenario = rgdp.append([remp, rsk, rsg, rsc, qr_tot, rsx, rsm, 
                            qr_agr, qr_ff, qr_elc, qr_eii, qr_oth, qr_con, qr_ser,
                           yre_agr, yre_ff, yre_elc, yre_eii, yre_oth, yre_con, yre_ser,
                           qex_agr, qex_ff, qex_elc, qex_eii, qex_oth, qex_con, qex_ser,
                           qem_agr, qem_ff, qem_elc, qem_eii, qem_oth, qem_con, qem_ser])
    
    # Add model version here
    scenario["Model"] = "E3ME_v1.0"
    # Add scenario name
    scenario["Scenario"] = str(m)
    
    # Calculate world region
    world = scenario.groupby(["Model", "Scenario","Category", "Variable", "Unit"]).sum().reset_index()
    world["Region"] = "World"
    scenario = scenario.append(world)
    
    # Calculate EU
    scenario["Region"] = np.where(scenario["Region"].isin(eu), "EU28", scenario["Region"])
    
    # Calculate REP
    scenario["Region"] = np.where(scenario["Region"].isin(rep), "REP", scenario["Region"])
    scenario = scenario.groupby(["Model", "Scenario","Category", "Variable", "Unit", "Region"]).sum().reset_index()

    
#     # Reorder cols and format to 5 year increments
    scenario = scenario.filter(["Model", "Scenario", "Region","Category", "Variable", "Unit", 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050])
    # Append to main results df
    results_pt2 = results_pt2.append(scenario)

scenario

Processing: SUP_NPi_Default


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:382: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([remp, rsk, rsg, rsc, qr_tot, rsx, rsm,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:396: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:409: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_pt2 = results_pt2.append(scenario)


Processing: SUP_2C_Default


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:382: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([remp, rsk, rsg, rsc, qr_tot, rsx, rsm,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:396: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:409: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_pt2 = results_pt2.append(scenario)


Processing: SUP_1p5C_Default


...

Processing: SUP_2C_VAT


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:382: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([remp, rsk, rsg, rsc, qr_tot, rsx, rsm,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:396: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:409: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_pt2 = results_pt2.append(scenario)


Processing: SUP_1p5C_VAT


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:382: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([remp, rsk, rsg, rsc, qr_tot, rsx, rsm,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:396: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\312892540.py:409: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_pt2 = results_pt2.append(scenario)


,Model,Scenario,Region,Category,Variable,Unit,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,E3ME_v1.0,SUP_1p5C_VAT,Argentina,Aggregate,Employment ( mn persons),million,16.861385,17.836715,17.447771,18.609094,19.642420,20.891644,22.821356,24.142582,24.969422
1,E3ME_v1.0,SUP_1p5C_VAT,Australia,Aggregate,Employment ( mn persons),million,11.271705,11.959770,12.205134,12.853460,13.455230,14.199933,15.077507,15.588736,15.871159
2,E3ME_v1.0,SUP_1p5C_VAT,Brazil,Aggregate,Employment ( mn persons),million,99.173383,103.897852,97.368469,103.327508,104.339656,104.388586,103.582430,101.527023,98.322812
3,E3ME_v1.0,SUP_1p5C_VAT,Canada,Aggregate,Employment ( mn persons),million,17.100070,18.372545,18.572951,19.184022,19.771268,20.514475,21.302080,21.948287,22.495076
4,E3ME_v1.0,SUP_1p5C_VAT,China,Aggregate,Employment ( mn persons),million,784.662500,862.514125,888.091250,831.937188,767.871500,729.843875,698.429062,671.995750,635.579938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,E3ME_v1.0,SUP_1p5C_VAT,Taiwan,Production by sector,Services,billion US$2010/yr,424.264188,482.794400,540.222957,581.148505,641.738898,727.076284,801.424874,754.670021,789.985948
1436,E3ME_v1.0,SUP_1p5C_VAT,Turkey,Production by sector,Services,billion US$2010/yr,519.958750,1512.237047,2587.749081,3294.324902,3345.548897,3385.055491,3398.068861,3393.284037,3398.665327
1437,E3ME_v1.0,SUP_1p5C_VAT,USA,Production by sector,Services,billion US$2010/yr,18364.670855,25379.444484,27947.250864,32634.210140,35299.444044,38507.190786,42110.790310,45238.198909,48552.104592
1438,E3ME_v1.0,SUP_1p5C_VAT,Ukraine,Production by sector,Services,billion US$2010/yr,139.050707,130.809715,134.035437,158.185770,183.697879,207.651087,239.813217,258.046680,279.493632


In [25]:
results_pt2.to_excel(r"Output/NAVIGATE/E3ME_task_26_sector_080923.xlsx")

### Results for inequality study

In [7]:
# Set list of mre files
# mre_list = ["SUP_NPi_Default", "SUP_1p5C_Default", "SUP_1p5C_Lab"]
mre_list = ["SUP_NPi_Default", "SUP_2C_Default", "SUP_1p5C_Default",
           "SUP_2C_Lab", "SUP_1p5C_Lab", "SUP_2C_Lump", "SUP_1p5C_Lump",
           "SUP_2C_VAT", "SUP_1p5C_VAT"]
# mre_list = ["SUP_NPi_Default", "SUP_2C_Default", "SUP_1p5C_Default"]
years = range(2010, 2051)
results = pd.DataFrame()

for m in mre_list:
    # Load mre
    print("Processing: " + str(m))
    mre = MRE("C:/E3ME/EEIST_NAVIGATE/Output/" + str(m) + ".mre")
    
    # 2d variables
    # Aggregate GDP (+ conversion)
    rgdp = pd.DataFrame((mre['RGDP'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rgdp["Variable"] = "GDP|MER"
    rgdp["Unit"] = "billion US$2010/yr"
    
    # Population (+ conversion)
    rpop = pd.DataFrame((mre['RPOP'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rpop["Variable"] = "Population"
    rpop["Unit"] = "million"
    
    # Aggregate gov consumption (+ conversion)
    rsg = pd.DataFrame((mre['RSK'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsg["Variable"] = "Investment"
    rsg["Unit"] = "billion US$2010/yr"
    
    # Aggregate investment (+ conversion)
    rsk = pd.DataFrame((mre['RSG'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsk["Variable"] = "Expenditure|Government"
    rsk["Unit"] = "billion US$2010/yr"
    
    # Aggregate household consumption (+ conversion)
    rsc = pd.DataFrame((mre['RSC'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsc["Variable"] = "Expenditure|Household"
    rsc["Unit"] = "billion US$2010/yr"
    
    # Aggregate export (+ conversion)
    rsx = pd.DataFrame((mre['RSX'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsx["Variable"] = "Export"
    rsx["Unit"] = "billion US$2010/yr"
    
    # Aggregate import (+ conversion)
    rsm = pd.DataFrame((mre['RSM'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsm["Variable"] = "Import"
    rsm["Unit"] = "billion US$2010/yr"
    
    # Aggregate employment (+ conversion)
    remp = pd.DataFrame((mre['REMP'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    remp["Variable"] = "Employment"
    remp["Unit"] = "million"    
    
    # Aggregate unemployment (+ conversion)
    rune = pd.DataFrame((mre['RUNE'][0]/1000), index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rune["Variable"] = "Unemployment"
    rune["Unit"] = "million"
    
    # Cabron price (+ conversion)
    repp = pd.DataFrame((mre['REPP'][0]/(44/12))*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    repp["Variable"] = "Price|Carbon"
    repp["Unit"] = "US$2010/t CO2"
    
    # Carbon reveunues
    rpin = pd.DataFrame((mre['RPIN'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rpin["Variable"] = "Revenue|government|Tax|Carbon Tax"
    rpin["Unit"] = "billion US$2010/yr"    
    
    # Carbon reveunues from hh
    rphh = pd.DataFrame((mre['RPHH'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rphh["Variable"] = "Revenue|government|Carbon Tax|Household"
    rphh["Unit"] = "billion US$2010/yr"         
    
    # Lump sum
    rsep = pd.DataFrame((mre['RSEP'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsep["Variable"] = "Expenditure|Government|Transfers to Household|Carbon Tax"
    rsep["Unit"] = "billion US$2010/yr"        
    
    # Subsidies
    rsub = pd.DataFrame((mre['RSUB'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    rsub["Variable"] = "Subsidies|Energy|Non-fossil w/ CCS"
    rsub["Unit"] = "billion US$2010/yr"     
    
    # EE
    reei = pd.DataFrame((mre['REEI'][0]/1000)*1.33, index=regions, columns=years).reset_index().rename(columns={"index":"Region"})
    reei["Variable"] = "Expenditure|Government|Other"
    reei["Unit"] = "billion US$2010/yr"     
    
    # 3d variables
    # CO2 emissions (+ conversion)
    fco2 = {}
    fco2_e = {}
    # Power generation
    mewk = {}
    mewk_tot = {}
    mewk_bccs = {}
    mewk_b = {}
    mewk_btot = {}
    mewk_cccs = {}
    mewk_c = {}
    mewk_ctot = {}
    mewk_gccs = {}
    mewk_g = {}
    mewk_gtot = {}
    mewk_hy = {}
    mewk_h = {}
    mewk_n = {}
    mewk_o = {}
    mewk_scsp = {}
    mewk_spv = {}
    mewk_stot = {}
    mewk_w = {}
    mewk_wof = {}
    mewk_won = {}
    mewk_geo = {}
    # Primary Energy
    rery = {}
    rery_tot = {}
    rery_c = {}
    rery_g = {}
    rery_o = {}
    rery_b = {}
    rery_s = {}
    rery_w = {}
    rery_geo = {}
    rery_h = {}
    rery_n = {}
    # Secondary Energy
    fret = {}
    
    for r, reg in enumerate(regions):
        fco2[reg] = pd.DataFrame((mre['FCO2'][r])/(3/11)/1000, index=fuel, columns=years)
        # Get just energy emissions
        fco2_e[reg] = fco2[reg].drop(["8 Ore-extra.(non-energy)", "23 Non-energy use"])
        
        # get sums of emissions across fuel users
        fco2[reg] = fco2[reg].sum()
        fco2_e[reg] = fco2_e[reg].sum()
        
        # MEWK
        mewk[reg] = pd.DataFrame(mre["MEWK"][r], index=power, columns=years)
        # Total electricty capacity
        mewk_tot[reg] = mewk[reg].sum()
        # Biomass w/ccs
        mewk_bccs[reg] = mewk[reg].iloc[[9,11,13]].sum()
        # Biomass
        mewk_b[reg] = mewk[reg].iloc[[8, 10, 12]].sum()
        # Biomass TOTAL
        mewk_btot[reg] = mewk[reg].iloc[[8,9,10,11,12,13]].sum()
    
        # Coal w/ccs
        mewk_cccs[reg] = mewk[reg].iloc[3]
        # Coal
        mewk_c[reg] = mewk[reg].iloc[2]
        # Coal TOTAL
        mewk_ctot[reg] = mewk[reg].iloc[[2,3]].sum()
        
        # Gas w/ccs
        mewk_gccs[reg] = mewk[reg].iloc[[5, 7]].sum()
        # Gas
        mewk_g[reg] = mewk[reg].iloc[[4, 6]].sum()
        # Gas TOTAL
        mewk_gtot[reg] = mewk[reg].iloc[[4,5,6,7]].sum()
        
        # Hydro
        mewk_hy[reg] = mewk[reg].iloc[[14, 15, 21]].sum()
        # Nuclear
        mewk_n[reg] = mewk[reg].iloc[0]
        # Oil 
        mewk_o[reg] = mewk[reg].iloc[1]
        # CSP 
        mewk_scsp[reg] = mewk[reg].iloc[19]
        # Solar pv
        mewk_spv[reg] = mewk[reg].iloc[18]
        # Solar TOTAL
        mewk_stot[reg] = mewk[reg].iloc[[18,19]].sum()

        # Total wind
        mewk_w[reg] = mewk[reg].iloc[[16, 17]].sum()
        # Offshore wind
        mewk_wof[reg] = mewk[reg].iloc[17]
        # Onshore wind
        mewk_won[reg] = mewk[reg].iloc[16]
        # Geothermal
        mewk_geo[reg] = mewk[reg].iloc[20]
        
        # RERY
        rery[reg] = pd.DataFrame((mre["RERY"][r])/1000, index=power_erti, columns=years)
        # Total electricty capacity
        rery_tot[reg] = rery[reg].sum()
        # Coal
        rery_c[reg] = rery[reg].iloc[[2]].sum()
        # Gas
        rery_g[reg] = rery[reg].iloc[[3]].sum()
        # Oil
        rery_o[reg] = rery[reg].iloc[[1]].sum()
        # Biomass
        rery_b[reg] = rery[reg].iloc[[4,5,6]].sum()
        # Solar
        rery_s[reg] = rery[reg].iloc[[11]].sum()
        # Wind
        rery_w[reg] = rery[reg].iloc[[9,10]].sum()
        # Geothermal
        rery_geo[reg] = rery[reg].iloc[[12]].sum()
        # Hydro
        rery_h[reg] = rery[reg].iloc[[7,8,13]].sum()        
        # Nuclear
        rery_n[reg] = rery[reg].iloc[[0]].sum()       
        
        # FRET
        fret[reg] = pd.DataFrame((mre['FRET'][r])*1000*4.1868E-08, index=fuel, columns=years)
        fret[reg] = fret[reg].sum()
        
    
    fco2 = pd.DataFrame.from_dict(fco2, orient='index').reset_index().rename(columns={"index":"Region"})
    fco2["Variable"] = "Emissions|CO2"
    fco2["Unit"] = "Mt CO2/yr"
    
    fco2_e = pd.DataFrame.from_dict(fco2_e, orient='index').reset_index().rename(columns={"index":"Region"})
    fco2_e["Variable"] = "Emissions|CO2|Energy"
    fco2_e["Unit"] = "Mt CO2/yr"
    
    mewk_tot = pd.DataFrame.from_dict(mewk_tot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_tot["Variable"] = "Capacity|Electricity"
    mewk_tot["Unit"] = "GW"
    
    mewk_bccs = pd.DataFrame.from_dict(mewk_bccs, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_bccs["Variable"] = "Capacity|Electricity|Biomass|w/ CCS"
    mewk_bccs["Unit"] = "GW"    

    mewk_b = pd.DataFrame.from_dict(mewk_b, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_b["Variable"] = "Capacity|Electricity|Biomass|w/o CCS"
    mewk_b["Unit"] = "GW"    

    mewk_btot = pd.DataFrame.from_dict(mewk_btot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_btot["Variable"] = "Capacity|Electricity|Biomass"
    mewk_btot["Unit"] = "GW"        
    
    mewk_cccs = pd.DataFrame.from_dict(mewk_cccs, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_cccs["Variable"] = "Capacity|Electricity|Coal|w/ CCS"
    mewk_cccs["Unit"] = "GW"    
 
    mewk_c = pd.DataFrame.from_dict(mewk_c, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_c["Variable"] = "Capacity|Electricity|Coal|w/o CCS"
    mewk_c["Unit"] = "GW"    

    mewk_ctot = pd.DataFrame.from_dict(mewk_ctot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_ctot["Variable"] = "Capacity|Electricity|Coal"
    mewk_ctot["Unit"] = "GW"        
    
    mewk_gccs = pd.DataFrame.from_dict(mewk_gccs, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_gccs["Variable"] = "Capacity|Electricity|Gas|w/ CCS"
    mewk_gccs["Unit"] = "GW" 

    mewk_g = pd.DataFrame.from_dict(mewk_g, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_g["Variable"] = "Capacity|Electricity|Gas|w/o CCS"
    mewk_g["Unit"] = "GW" 
    
    mewk_gtot = pd.DataFrame.from_dict(mewk_gtot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_gtot["Variable"] = "Capacity|Electricity|Gas"
    mewk_gtot["Unit"] = "GW"     
    
    mewk_hy = pd.DataFrame.from_dict(mewk_hy, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_hy["Variable"] = "Capacity|Electricity|Hydro"
    mewk_hy["Unit"] = "GW"    
    
    mewk_n = pd.DataFrame.from_dict(mewk_n, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_n["Variable"] = "Capacity|Electricity|Nuclear"
    mewk_n["Unit"] = "GW"        
    
    mewk_o = pd.DataFrame.from_dict(mewk_o, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_o["Variable"] = "Capacity|Electricity|Oil|w/o CCS"
    mewk_o["Unit"] = "GW"   
    
    mewk_scsp = pd.DataFrame.from_dict(mewk_scsp, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_scsp["Variable"] = "Capacity|Electricity|Solar|CSP"
    mewk_scsp["Unit"] = "GW"       
    
    mewk_spv = pd.DataFrame.from_dict(mewk_spv, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_spv["Variable"] = "Capacity|Electricity|Solar|PV"
    mewk_spv["Unit"] = "GW"        

    mewk_stot = pd.DataFrame.from_dict(mewk_stot, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_stot["Variable"] = "Capacity|Electricity|Solar"
    mewk_stot["Unit"] = "GW"    
    
    mewk_w = pd.DataFrame.from_dict(mewk_w, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_w["Variable"] = "Capacity|Electricity|Wind"
    mewk_w["Unit"] = "GW"        

    mewk_wof = pd.DataFrame.from_dict(mewk_wof, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_wof["Variable"] = "Capacity|Electricity|Wind|Offshore"
    mewk_wof["Unit"] = "GW"            
    
    mewk_won = pd.DataFrame.from_dict(mewk_won, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_won["Variable"] = "Capacity|Electricity|Wind|Onshore"
    mewk_won["Unit"] = "GW"           
    
    mewk_geo = pd.DataFrame.from_dict(mewk_geo, orient='index').reset_index().rename(columns={"index":"Region"})
    mewk_geo["Variable"] = "Capacity|Electricity|Geothermal"
    mewk_geo["Unit"] = "GW"
    
    rery_tot = pd.DataFrame.from_dict(rery_tot, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_tot["Variable"] = "Primary Energy"
    rery_tot["Unit"] = "EJ/yr"    
    
    rery_c = pd.DataFrame.from_dict(rery_c, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_c["Variable"] = "Primary Energy|Coal"
    rery_c["Unit"] = "EJ/yr"     

    rery_g = pd.DataFrame.from_dict(rery_g, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_g["Variable"] = "Primary Energy|Gas"
    rery_g["Unit"] = "EJ/yr"    
    
    rery_o = pd.DataFrame.from_dict(rery_o, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_o["Variable"] = "Primary Energy|Oil"
    rery_o["Unit"] = "EJ/yr"
    
    rery_b = pd.DataFrame.from_dict(rery_b, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_b["Variable"] = "Primary Energy|Biomass"
    rery_b["Unit"] = "EJ/yr"
    
    rery_s = pd.DataFrame.from_dict(rery_s, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_s["Variable"] = "Primary Energy|Solar"
    rery_s["Unit"] = "EJ/yr"
    
    rery_w = pd.DataFrame.from_dict(rery_w, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_w["Variable"] = "Primary Energy|Wind"
    rery_w["Unit"] = "EJ/yr"
    
    rery_geo = pd.DataFrame.from_dict(rery_geo, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_geo["Variable"] = "Primary Energy|Geothermal"
    rery_geo["Unit"] = "EJ/yr"    
    
    rery_h = pd.DataFrame.from_dict(rery_h, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_h["Variable"] = "Primary Energy|Hydro"
    rery_h["Unit"] = "EJ/yr" 
    
    rery_n = pd.DataFrame.from_dict(rery_n, orient='index').reset_index().rename(columns={"index":"Region"})
    rery_n["Variable"] = "Primary Energy|Nuclear"
    rery_n["Unit"] = "EJ/yr" 
    
    fret = pd.DataFrame.from_dict(fret, orient='index').reset_index().rename(columns={"index":"Region"})
    fret["Variable"] = "Secondary Energy|Electricity"
    fret["Unit"] = "EJ/yr"
    
    # Merge variables
    scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
                            mewk_btot, mewk_cccs, mewk_c, mewk_ctot, mewk_gccs, mewk_g, mewk_gtot, mewk_hy, mewk_n,
                            mewk_o, mewk_scsp, mewk_spv, mewk_stot, mewk_w, mewk_wof, mewk_won, mewk_geo, rery_tot,
                           rery_c, rery_g, rery_o, rery_b, rery_s, rery_w, rery_geo, rery_h, rery_n, fret,
                           rpin,rphh,rsep,rsub,reei])
    
    # Add model version here
    scenario["Model"] = "E3ME v1.0"
    # Add scenario name
    scenario["Scenario"] = str(m)
    # Correct scenario name
    scenario["Scenario"] = np.where(scenario["Scenario"]=="SUP_1p5C_Dif10", "SUP_1p5C_Regional", scenario["Scenario"])
    scenario["Scenario"] = np.where(scenario["Scenario"]=="SUP_2C_Dif10", "SUP_2C_Regional", scenario["Scenario"])
    # Calculate world region
    world = scenario.groupby(["Model", "Scenario", "Variable", "Unit"]).sum().reset_index()
    world["Region"] = "World"
    # Get rid of carbon price
    world = world.loc[world["Variable"]!="Price|Carbon"]
    scenario = scenario.append(world)
    # Reorder cols and format to 5 year increments
    scenario = scenario.filter(["Model", "Scenario", "Region", "Variable", "Unit", 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050])
    # Append to main results df
    results = results.append(scenario)
    
scenario

Processing: SUP_NPi_Default


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:350: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:372: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


Processing: SUP_2C_Default


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:350: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:372: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


Processing: SUP_1p5C_Default


...

Processing: SUP_2C_VAT


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:350: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:372: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


Processing: SUP_1p5C_VAT


C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:350: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = rgdp.append([rsg, rpop, rsk, rsc, rsx, rsm, remp, rune, repp, fco2, fco2_e, mewk_tot, mewk_bccs, mewk_b,
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scenario = scenario.append(world)
C:\Users\CL852\AppData\Local\Temp\ipykernel_19316\2392675200.py:372: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(scenario)


,Model,Scenario,Region,Variable,Unit,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,E3ME v1.0,SUP_1p5C_VAT,Belgium,GDP|MER,billion US$2010/yr,482.976325,515.173422,510.728811,595.677241,622.920587,639.076970,667.139222,709.144695,745.797999
1,E3ME v1.0,SUP_1p5C_VAT,Denmark,GDP|MER,billion US$2010/yr,323.410136,344.802791,367.907135,429.799226,479.169449,516.405833,556.066516,603.085341,652.194012
2,E3ME v1.0,SUP_1p5C_VAT,Germany,GDP|MER,billion US$2010/yr,3410.652000,3717.737363,3762.749218,4294.610233,4587.528435,4737.816773,4886.777105,5226.469745,5554.201363
3,E3ME v1.0,SUP_1p5C_VAT,Greece,GDP|MER,billion US$2010/yr,300.621874,245.748796,232.231051,283.605251,282.707169,292.365961,304.531451,331.432135,358.660892
4,E3ME v1.0,SUP_1p5C_VAT,Spain,GDP|MER,billion US$2010/yr,1426.703136,1424.853106,1390.851656,1719.842285,1858.407005,1999.389998,2154.304075,2306.248595,2444.863356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,E3ME v1.0,SUP_1p5C_VAT,World,Revenue|government|Carbon Tax|Household,billion US$2010/yr,0.000000,3.828644,9.098209,91.768291,149.201767,158.570220,165.956570,192.333209,256.004725
44,E3ME v1.0,SUP_1p5C_VAT,World,Revenue|government|Tax|Carbon Tax,billion US$2010/yr,0.000000,30.474479,64.059797,1504.721540,2511.676517,2992.625370,3201.184892,3572.097872,4086.702712
45,E3ME v1.0,SUP_1p5C_VAT,World,Secondary Energy|Electricity,EJ/yr,66.625303,74.887626,84.608191,93.786452,110.633330,128.260015,148.874804,171.827748,191.221666
46,E3ME v1.0,SUP_1p5C_VAT,World,Subsidies|Energy|Non-fossil w/ CCS,billion US$2010/yr,0.000000,139.688097,231.849182,767.894398,685.810559,962.802839,1030.238824,661.591907,278.138963


In [21]:
results.to_excel(r"macroMIP_T2.6_inequality_E3ME.xlsx")